In [120]:
import pandas as pd
import matplotlib.pyplot as plt
import cufflinks as cf
import numpy as np
from sklearn.impute import SimpleImputer
from scipy.stats import ks_2samp


In [121]:
pd.set_option("display.max_columns",30)
pd.set_option("display.max_rows",3000)
cf.go_offline()


In [123]:
def completitud_datos_nulos(df): ...
def complete_continuous_variables(df, col, strategy='median'): ...


In [126]:
complete_continuous_variables(df, ["body"])


In [127]:
varc = list(df.describe())
varc = [x for x in varc if x not in 'survived']
vard = [x for x in df.columns if x not in varc+['survived']]
tgt = 'survived'



In [128]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = df[varc].copy()
y = df[tgt].astype(int)
sc = MinMaxScaler()
Xs = pd.DataFrame(sc.fit_transform(X), columns=varc)
Xt, Xv, yt, yv = train_test_split(Xs, y, train_size=0.7)


In [129]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

def entrenar(param, modelo, X, y): ...
def metricas(Xt, Xv, yt, yv, modelo): ...
def evaluar_precision(Xv, yv, modelo): ...


In [106]:

df_categoricas = pd.get_dummies(df[['sex', 'pclass', 'embarked']], drop_first=True)


Xs_full = pd.concat([Xt_imputed, df_categoricas], axis=1)


Xs_full.isnull().sum()


,0
pclass,393
age,393
sibsp,393
parch,393
fare,393
body,393
pclass,0
sex_male,0
embarked_Q,0
embarked_S,0


In [130]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

def entrenar(param, modelo, X, y): ...
def metricas(Xt, Xv, yt, yv, modelo): ...
def evaluar_precision(Xv, yv, modelo): ...


In [137]:
from sklearn.model_selection import RandomizedSearchCV

def entrenar(param, modelo, X, y):
    grid = RandomizedSearchCV(
        param_distributions=param,
        n_jobs=-1,
        n_iter=10,
        cv=4,
        estimator=modelo,
        error_score='raise'
    )
    grid.fit(X, y)
    return grid, grid.best_estimator_, grid.best_score_, grid.best_params_


In [141]:
param_mlpc = dict(
    hidden_layer_sizes=[(50,), (100,), (50, 50)],
    activation=['relu', 'tanh'],
    solver=['adam'],
    alpha=[0.0001, 0.001],
    learning_rate=['adaptive']
)


In [61]:
import pickle
with open('./redneural.pkl', 'wb') as file:
    pickle.dump(modelo_mlpc, file)


In [62]:
from sklearn.metrics import accuracy_score

def evaluar_precision(Xv, yv, modelo):
    y_pred = modelo.predict(Xv)
    accuracy = accuracy_score(yv, y_pred)
    print(f"Precisión (accuracy) en validación: {accuracy:.2%}")
    return accuracy


In [69]:
from sklearn.metrics import roc_auc_score

def metricas(Xt, Xv, yt, yv, modelo):
    d = {
        'train': round(roc_auc_score(yt, modelo.predict_proba(Xt)[:, 1]), 3),
        'validate': round(roc_auc_score(yv, modelo.predict_proba(Xv)[:, 1]), 3)
    }
    return d


In [71]:
from sklearn.metrics import accuracy_score

def evaluar_precision(Xv, yv, modelo):
    y_pred = modelo.predict(Xv)
    accuracy = accuracy_score(yv, y_pred)
    print(f"Precisión (accuracy) en validación: {accuracy:.2%}")
    return accuracy


In [73]:
Xt.isnull().sum().sort_values(ascending=False)


,0
body,825
age,186
pclass,0
sibsp,0
parch,0
fare,0


In [74]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy="median")  # o "mean"
Xt_imputed = pd.DataFrame(imp.fit_transform(Xt), columns=Xt.columns)
Xv_imputed = pd.DataFrame(imp.transform(Xv), columns=Xv.columns)


In [144]:
modelo_mlpc = MLPClassifier(max_iter=1000, random_state=42)
modelo_mlpc, best_estimator_mlpc, score_mlpc, params_mlpc = entrenar(param_mlpc, modelo_mlpc, Xt_imputed, yt)
evaluar_precision(Xv_imputed, yv, modelo_mlpc)



In [77]:
modelo_rf = RandomForestClassifier()
modelo_rf, best_estimator_rf, score_rf, params_rf = entrenar(param, modelo_rf, Xt_imputed, yt)
evaluar_precision(Xv_imputed, yv, modelo_rf)


Precisión (accuracy) en validación: 59.80%


0.5979643765903307

In [80]:
param_svc = dict(
    C=np.arange(0.1, 2, 0.1),
    kernel=['linear', 'poly', 'rbf', 'sigmoid'],
    degree=range(2, 5),
    gamma=['scale', 'auto'],
    probability=[True]
)


In [81]:
modelo_svc = SVC(probability=True)
modelo_svc, best_estimator_svc, score_svc, params_svc = entrenar(param_svc, modelo_svc, Xt_imputed, yt)
evaluar_precision(Xv_imputed, yv, modelo_svc)


Precisión (accuracy) en validación: 67.94%


0.6793893129770993

In [90]:
param_adab = dict(
    n_estimators=range(50, 150, 25),
    learning_rate=np.arange(0.1, 1.1, 0.1),
    algorithm=['SAMME']  # Cambiar de 'SAMME.R' a 'SAMME'
)



In [91]:
modelo_adab = AdaBoostClassifier()
modelo_adab, best_estimator_adab, score_adab, params_adab = entrenar(param_adab, modelo_adab, Xt_imputed, yt)
evaluar_precision(Xv_imputed, yv, modelo_adab)


Precisión (accuracy) en validación: 70.99%


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning:

The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.



0.7099236641221374